In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Loading  Amazon UK product dataset which provides information on product categories,
# brands, prices, ratings, and more from from Amazon UK.
df = pd.read_csv(r"C:\Users\Tijana&Ivan\Desktop\IRON_HACK_PT_DATA_ANALYTICS\Week_13\Labs\archive\amz_uk_price_prediction_dataset.csv")

In [ ]:
# Filtering out columns in the dataframe where more than 80% of the values are missing
df = df[df.columns[df.isnull().mean() < 0.8]]

In [ ]:
# Checking for missing data
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Removing the "Id's" columns from the dataframe
df.drop(["uid","asin"], inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical = df.select_dtypes("number").loc[:, df.select_dtypes("number").nunique() < 20]
potential_categorical_from_numerical

In [ ]:
# Extracting columns with object (typically string) data types to create a categorical dataframe
# For demonstration purposes, let's consider the columns in potential_categorical_from_numerical
# as categorical variables.
df_categorical = pd.concat([df.select_dtypes("object"), potential_categorical_from_numerical], axis=1)

# Adjusting the numerical dataframe by removing the moved columns
df_numerical = df.select_dtypes("number").drop(columns=potential_categorical_from_numerical.columns)

In [ ]:
df_categorical

In [ ]:
df_numerical

In [ ]:
# Create a crosstab between the product category and the isBestSeller status.
# Generating a crosstab for 'category' and 'isBestSeller'
crosstab_result = pd.crosstab(df['category'], df['isBestSeller'])
crosstab_result
new_crosstab_result= crosstab_result.copy()

In [ ]:
crosstab_result["Total"] = crosstab_result[False]+crosstab_result[True]
crosstab_result["Proportion_BestSellers"] = round(crosstab_result[True]/crosstab_result["Total"],3)
sorted_crosstab= crosstab_result.sort_values(by = "Proportion_BestSellers", ascending= False)
sorted_crosstab

Are there categories where being a best-seller is more prevalent?

Health & Personal Care, Grocey and Smart Home Security & Lighting  has the highest proportion of best sellers with approx 5.8%.
Mobile Phone Accessories with 4.8% has also reltively high proportion of best_sellers while, 
for example Equestrian Sports Boots,Radio Communication, Climbing Footwear and many more does not have any best seler proportion indicating that no products from these categories have best seller status.




Statistical Tests:

Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.

In [ ]:
from scipy.stats import chi2_contingency

# Chi-square test for 'category' and 'isBestSeller'
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result)

chi2_statistic, chi2_p_value

A large value of the Chi- statistic suggests a significant association between the variables. 
P-value suggest strong relationship.

In [ ]:
from scipy.stats.contingency import association

# Computing the association between variables in 'crosstab_result' using the "cramer" method
association(new_crosstab_result, method="cramer")

The Cramér's V value of 0.122, suggests a weak association between the variables.
The strenght of this relationship is not very strong in practical terms.

In [ ]:
top_5_data = sorted_crosstab.iloc[0:10,:2]
top_5_data

In [ ]:
# Plotting a stacked bar chart for the 'crosstab_result' data
top_5_data.plot(kind="bar", stacked=True)
plt.show()

In [ ]:
# Plotting a grouped bar chart
px.histogram(df, x='category', color='isBestSeller', barmode='group')

Part 2: Exploring Product Prices and Ratings Across Categories and Brands
Objective: Investigate how different product categories influence product prices.

Preliminary Step: Remove outliers in product prices.

For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.

Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.



Box Plots:
Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
Which category tends to receive the highest median rating from customers? Don't filter here by top categories.


In [ ]:
df.shape

In [ ]:
# calculatin IGR(interquartile_range) of price
quantile_1= df["price"].quantile(0.25)
quantile_3=df["price"].quantile(0.75)
interquartile_range= quantile_3-quantile_1

In [ ]:
# finding and removing(filtering) Outliers
q1_outliers= quantile_1 - 1.5 * interquartile_range
q3_outliers= quantile_3 + 1.5 * interquartile_range

filtered_df= df[(df['price'] >= q1_outliers) & (df['price'] <= q3_outliers)]

In [ ]:
filtered_df.shape

Violin Plots:

Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.
Which product category tends to have the highest median price? Don't filter here by top categories.

In [ ]:
top_20_categories= filtered_df['category'].value_counts().nlargest(20).index

In [ ]:
print("Top 20 Categories:", top_20_categories)

In [ ]:
# Filter the DataFrame
top_categories_df = filtered_df[filtered_df['category'].isin(top_20_categories)]
print("Filtered DataFrame Size:", top_categories_df.shape)  

In [ ]:
print("Categories in Filtered DataFrame:", top_categories_df['category'].unique())  

In [ ]:
# Plotting a violin plot to visualize the distribution of price across different product categories
sns.violinplot(data=top_categories_df, x='price', y='category', hue = "category", legend = False, palette="coolwarm")
plt.show()

Looking at Violinplot, I would say that category "Men" has the highest median price.

Bar Charts:

Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
Which product category commands the highest average price? Don't filter here by top categories.


In [ ]:
# Calculating average price per category
average_price= filtered_df.groupby('category')['price'].mean().sort_values(ascending=False)

In [ ]:
top10_avg_price=average_price.nlargest(10)
top10_avg_price

In [ ]:
top10_avg_price_df = top10_avg_price.reset_index()
top10_avg_price_df.columns = ['category', 'price']

In [ ]:
# Plotting a bar plot to visualize the average 'price' for each 'category', using the "coolwarm" color palette
px.histogram(top10_avg_price_df, x='price', y='category')

The highest average price comes from Motherboards and it is 68.77

Box Plots:
Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
Which category tends to receive the highest median rating from customers? Don't filter here by top categories.

In [ ]:
top_categories_by_count = df['category'].value_counts().nlargest(10).index
df_top_categories_by_count = df[df['category'].isin(top_categories_by_count)]
df_top_categories_by_count

In [ ]:
px.box(df_top_categories_by_count, x='stars', y='category')

Based on theBox Plot Hair Care, Beauty, Bath&Body and Skin Care have the highest median rating from customers of 4.4

Objective: Analyze how product ratings (stars) correlate with product prices.

Correlation Coefficients:

Calculate the correlation coefficient between price and stars.
Is there a significant correlation between product price and its rating?

In [ ]:
filtered_df.head(1)

In [ ]:
# correlation coefficient between product price and ratings
correlation = filtered_df['price'].corr(filtered_df['stars'])
print(f"Correlation coefficient between price and rating: {correlation}")

-0.0777 indicates small or no correlation between variables.

**Visualisation:**
Scatter plot

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(filtered_df['stars'], filtered_df['price'], alpha=0.5)
plt.xlabel('Product Rating')
plt.ylabel('Product Price')
plt.title('Scatter Plot of Product Price vs Rating')
plt.show()

There is no trend that shows any correlation between price and rating

**Correlation Heatmap**

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(df_numerical.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

 **QQ Plot for Normality Check**

In [ ]:
import scipy.stats as stats

plt.figure(figsize=(8, 6))
stats.probplot(filtered_df['price'], dist="norm", plot=plt)
plt.title('QQ Plot for Product Prices')
plt.show()